### JaggedTensor and GridBatch
There are two fundamental classes in fVDB you will encounter frequently: `JaggedTensor` and `GridBatch`.  

```python
fvdb.JaggedTensor
fvdb.GridBatch
```

#### GridBatch

A `GridBatch` is an indexing structure which maps 3D `ijk` coordinates to integer offsets which can be used to look up attributes in a tensor.  The figure below illustrates this process for a GridBatch containing a single grid.


<center>

<img src="img/gridbatch.png"  alt="Image Index Grid" width="800"/>>

</center>

In practice, `GridBatch` is an ordered collection of 3D grids.  Specifically these 3D grids are **NanoVDB** grids of the special `IndexGrid` type which only stores a unique 'index' integer value at each active voxel location.  This 'index' is an offset into some external data array, a tensor, (one that is not contained in the `IndexGrid`/`GridBatch`) where contiguous array members correspond to spatially nearby voxels.  `IndexGrid` will allow us to reference into this 'sidecar' tensor of attribute data given a spatial `ijk` set of coordinates.

Each `IndexGrid` member in a `GridBatch` can have different topologies, different numbers of active voxels and different voxel dimensions and origins per-grid.

<center>

<img src="img/gridbatch_concept.svg"  alt="Image Index Grid" width="800"/>>

</center>

##### Images as 2D GridBatch


To help explain these concepts, let's consider how we might treat image data with this framework (an image can be thought of as a dense 2D grid after all).  If an image is a 2D grid of pixels, we could imagine the position of each pixel could be expressed as `i,j` coordinates.  For an RGB image, each pixel would contain 3 associated values (R,G,B).  

In this way, we could decompose an image into an `IndexGrid` of `i,j` coordinates and flatten the RGB values into a list of size `[Number of Pixels, 3]` (the features for our attribute tensor).  The `i,j` coordinates can be used with the `IndexGrid` to index into the list of RGB values to retrieve the RGB value for each pixel.  

<center>

<img src="img/image_index_grid_diagram.svg"  alt="Image Index Grid" width="1000"/>>

</center>

If we had many images, each of different sizes, we can imagine constructing a `GridBatch` as an ordered set of their `IndexGrid`s.  All the RGB values would go into a sidecar attribute tensor of *jagged* features where each feature array would have to be of a different length corresponding to the image size.  This is the essence of the relationship between `GridBatch` and `JaggedTensor` in fVDB, but more on `JaggedTensor` later…

<center>

<img src="img/grid_batch_diagram.svg"  alt="Grid Batch" width="600"/>>

</center>

Lastly, it is important to know that each grid in the `GridBatch` will be on the same device and processed together by operators in mini-batch-like fashion.  

Let's put together our first `GridBatch` to see how it works.

In [1]:
# import the usual suspects and fvdb
import numpy as np
import torch
import fvdb

In [2]:
batch_size = 8
# Randomly generate different numbers of voxels we desire in each grid in our batch
num_voxels_per_grid = [np.random.randint(100, 1_000) for _ in range(batch_size)]

# A list of randomly generated 3D indices for each grid in our batch in the range [-512, 512]
ijks = [torch.randint(-512, 512, (num_voxels_per_grid[i], 3), device='cuda') for i in range(batch_size)]

# Create an fvdb.GridBatch from the list of indices!!
grid = fvdb.sparse_grid_from_ijk(fvdb.JaggedTensor(ijks), # We'll explain JaggedTensor in a moment…
                                 voxel_sizes = [np.random.rand(3) for _ in range(batch_size)], # Random, different voxel sizes for each grid in our batch
                                 origins     = [np.random.rand(3) for _ in range(batch_size)], # Random, different grid origins for each grid in our batch
                                )

# This grid will be on the GPU because the `ijks` were on that device
assert(grid.device == ijks[0].device == torch.device('cuda:0'))

# Each member of the batch has different voxel size dimensions, a different origin in space and different number of voxels
for i in range(grid.grid_count):
    print(f"""Grid {i} has {grid.num_voxels_at(i)} voxels,
        voxel size of {grid.voxel_size_at(i).tolist()}
        and an origin of {grid.origin_at(i).tolist()}""")

# NOTE: This GridBatch (Batch of IndexGrids) just expresses the topology of the grids and can be used to reference a sidecar flat array of features but we won't create this sidecar in this example…
# We can get the index into this hypothetical sidecar feature array with any `ijk` coordinate (if we ask for an `ijk` not in the grid, -1 is given as the index)

# Let's retrieve a random ijk coordinate from each of the lists we used to make the grids
ijk_queries = fvdb.JaggedTensor([ijks[n][np.random.randint(len(ijks[n]))][None,:] for n  in range(grid.grid_count)])
# Use the GridBatch to get indices into the sidecar feature array from the `ijk` coordinate in each grid
feature_indices = grid.ijk_to_index(ijk_queries)
for i, (ijk,i_f) in enumerate(zip(ijk_queries, feature_indices)):
    print(f"Grid {i}, feature Index at ijk {ijk.jdata.tolist()} : {i_f.jdata.item()}")


Grid 0 has 565 voxels, 
        voxel size of [0.4180232286453247, 0.9078353047370911, 0.7569453120231628] 
        and an origin of [0.053342342376708984, 0.666057825088501, 0.5568195581436157]
Grid 1 has 505 voxels, 
        voxel size of [0.4231018126010895, 0.06702223420143127, 0.027880029752850533] 
        and an origin of [0.23663270473480225, 0.9503206014633179, 0.9787915945053101]
Grid 2 has 840 voxels, 
        voxel size of [0.552473783493042, 0.6610133647918701, 0.17690575122833252] 
        and an origin of [0.19191621243953705, 0.4792993664741516, 0.8062528967857361]
Grid 3 has 192 voxels, 
        voxel size of [0.3558811545372009, 0.006604234222322702, 0.05409170687198639] 
        and an origin of [0.7635682821273804, 0.5538259744644165, 0.6306257247924805]
Grid 4 has 374 voxels, 
        voxel size of [0.21053555607795715, 0.037602122873067856, 0.6572714447975159] 
        and an origin of [0.07630939781665802, 0.2442461997270584, 0.3808997869491577]
Grid 5 has 707 vo

There are many more convenient ways that fVDB provides to create a `GridBatch` besides building from lists of coordinates.  Please see the fVDB documentation for more useful examples for creating a `GridBatch` from pointclouds, meshes or dense tensors, to name a few.

#### JaggedTensor

`JaggedTensor` is the supporting feature data that is paired with a `GridBatch`.  

You can think of `JaggedTensor` as an ordered list of PyTorch Tensors, one for each grid in the `GridBatch`.  Same as `GridBatch`, the Tensors are all on the same device and processed together in a mini-batch. 

In [3]:
# From a GridBatch a matching JaggedTensor can be created from a Tensor of shape `(total_voxels, feature_dim)`
#   Here we create a JaggedTensor with 8 features (an arbitrary choice of number of features for our example) of random values for our GridBatch
features = grid.jagged_like(torch.randn(grid.total_voxels, 8, device=grid.device))

# This would have the equivalent effect:
features  = fvdb.JaggedTensor([torch.randn(grid.num_voxels[i], 8, device=grid.device) for i in range(grid.grid_count)])

In the `JaggedTensor` above, you can see how we constructed it with a list of heterogeneously sized Tensors whose shapes were of the form
`[ [B1, C], [B2, C], [B3, C], …] ]` where the value of `B[#]` would be the number of active voxels in each grid of the `GridBatch` and `C` is the number of feature channels (8 was chosen in this case).

Note how each Tensor element in our `JaggedTensor` can have different numbers of active voxels (similar to `GridBatch`) but the same number of per-voxel feature channels.  This is distinctly different from the classic representation of 3D data in a PyTorch `Tensor` which is usually a homogeneously shaped Tensor of shape `[B, C, H, W, D]` where `B` would be the number of "grids" in our batch, `C` is the number of feature channels, and `H, W, D` are the 3 index dimensions of a dense grid.

Internally, `JaggedTensor` does not represent the list of features as a list of differently sized PyTorch tensors, but instead stores a single tensor of the features accompanied by a special index structure that allows us to access the feature data for each grid in the `GridBatch`.  

The `jdata` attribute of a `JaggedTensor` contains all the feature data values in this single list.  `jdata` is a `Tensor` of shape `[N, C]` where `N` is the total number of active voxels in the batch and `C` is the number of feature channels.  

`jdata`'s shape would be equivalent to the result of concatenating the list of heterogeneously sized Tensors, mentioned above, along their first axis into a single Tensor whose shape would be `[B1+B2+B3…+Bn, C]`.

<center>

<img src = "img/jdata.jpg" width=1200 alt="jdata">

</center>

In [4]:
print(f"size of features data for the entire GridBatch: {features.jdata.shape}")

size of features data for the entire GridBatch: torch.Size([3958, 8])


To determine which grid each feature belongs to, `JaggedTensor` contains indexing information in its `jidx` attribute.

`jidx` is a `Tensor` of shape `[N]` where `N` is again the total number of active voxels across the batch.  Each element of `jidx` is an integer that tells us which grid in the `GridBatch` the corresponding feature in `jdata` belongs to.  The grid membership in `jidx` is ordered starting from 0 and members of the same batch are contiguous.

<center>

<img src = "img/jidx.jpg" width=1200 alt="jdata">

</center>

In [5]:
print(f"per-feature membership for each voxel in the GridBatch: {features.jidx}")
print(f"\nthe size of the features of the 4th grid in this batch: {features.jdata[features.jidx==3].shape}")

per-feature membership for each voxel in the GridBatch: tensor([0, 0, 0,  ..., 7, 7, 7], device='cuda:0', dtype=torch.int16)

the size of the features of the 4th grid in this batch: torch.Size([192, 8])


Additionally, `JaggedTensor` has a `joffsets` attribute that can also be used to index into `jdata` to get the feature data for each grid in the batch.

The `joffsets` attribute is a `Tensor` of shape `[B, 2]` which has the start and end offset into `jdata` that corresponds to each grid in the batch.  This is essentially the same information that can be found in `jidx` but expressed in a different form.

<center>

<img src = "img/joffsets.jpg" width=1200 alt="jdata">

</center>

In [6]:
print("per-grid offsets into the feature data:")
print(features.joffsets)
print(f"\nthe size of the features of the 4th grid in this batch: {features.jdata[features.joffsets[3][0]:features.joffsets[3][1]].shape}")

per-grid offsets into the feature data:
tensor([[   0,  565],
        [ 565, 1070],
        [1070, 1910],
        [1910, 2102],
        [2102, 2476],
        [2476, 3183],
        [3183, 3345],
        [3345, 3958]], device='cuda:0')

the size of the features of the 4th grid in this batch: torch.Size([192, 8])


The features stored in a `JaggedTensor` can be of any type that PyTorch supports, including float, float64, float16, int, bool, etc., and we can have an arbitrary number of feature channels per voxel.  

For instance, there could be a `JaggedTensor` with 1 float feature that represents a signed distance field in each grid, or 3 float features that represent some RGB color in each voxel of the grids, or a 192 float feature that represents a learned feature vector of each voxel in each grids.

In [7]:
# A single scalar feature
features = grid.jagged_like(torch.randn(grid.total_voxels, 1, dtype=torch.float, device=grid.device))

# Cast to a double
features = features.double()

# A JaggedTensor of 192 float features
features = grid.jagged_like(torch.randn(grid.total_voxels, 192, dtype=torch.float, device=grid.device))